In [31]:
# 트리베이스의 부스팅이 성능이 제일 좋음.
# 배깅은 동시에 진행, 부스팅은차례대로 돌림, 앞단계에 실행한 결과를 보고 새로운 알고리즘을 실행하여 더 높은 성능을 보임, but 시간이 오래걸린다는 단점 존재
# 배깅의 대표적인 것은 랜덤포레스트,
# 알고리즘의 대표특성정도는 알아두자
#

#랜덤포레스트
- 배깅

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

###1. 데이터 준비

In [33]:
# !wget https://raw.githubusercontent.com/devdio/flyai_datasets/main/winequalityN.csv

# 코랩이 내부적으로 리눅스라서 위 명령어가 되는거임


In [34]:
!gdown https://raw.githubusercontent.com/devdio/flyai_datasets/main/winequalityN.csv

# 이건 윈도우에서 사용가능한 명령어
# 구글드라이브에서 대용량의 데이터셋을 받을 때도 유용
# !pip install gdown 해주면 됨

Downloading...
From: https://raw.githubusercontent.com/devdio/flyai_datasets/main/winequalityN.csv
To: /content/winequalityN.csv
390kB [00:00, 77.0MB/s]        


In [35]:
# 데이터 로딩
wine= pd.read_csv('winequalityN.csv')
wine.shape

(6497, 13)

In [36]:
wine.head()
#
df=wine.copy()
df['type'].unique()

array(['white', 'red'], dtype=object)

In [37]:
df['type'].value_counts()
# 이 방법 추천
# 이진 분류시에는 두 분류의 수를 비슷하게 맞춤 레드를 더 늘려야겠네,
# 지금은 다중이니까 퀄리티 변수 사용


white    4898
red      1599
Name: type, dtype: int64

In [38]:
df['quality'].value_counts().sort_index()
# sort_index로 나열하기


3      30
4     216
5    2138
6    2836
7    1079
8     193
9       5
Name: quality, dtype: int64

In [39]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
fixed acidity,6487.0,7.216579,1.296750,3.80000,6.40000,7.00000,7.70000,15.90000
volatile acidity,6489.0,0.339691,0.164649,0.08000,0.23000,0.29000,0.40000,1.58000
citric acid,6494.0,0.318722,0.145265,0.00000,0.25000,0.31000,0.39000,1.66000
residual sugar,6495.0,5.444326,4.758125,0.60000,1.80000,3.00000,8.10000,65.80000
chlorides,6495.0,0.056042,0.035036,0.00900,0.03800,0.04700,0.06500,0.61100
free sulfur dioxide,6497.0,30.525319,17.749400,1.00000,17.00000,29.00000,41.00000,289.00000
total sulfur dioxide,6497.0,115.744574,56.521855,6.00000,77.00000,118.00000,156.00000,440.00000
density,6497.0,0.994697,0.002999,0.98711,0.99234,0.99489,0.99699,1.03898
pH,6488.0,3.218395,0.160748,2.72000,3.11000,3.21000,3.32000,4.01000
sulphates,6493.0,0.531215,0.148814,0.22000,0.43000,0.51000,0.60000,2.00000


In [40]:
# 결측치
df.isna().sum(axis=0)

type                     0
fixed acidity           10
volatile acidity         8
citric acid              3
residual sugar           2
chlorides                2
free sulfur dioxide      0
total sulfur dioxide     0
density                  0
pH                       9
sulphates                4
alcohol                  0
quality                  0
dtype: int64

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6497 non-null   object 
 1   fixed acidity         6487 non-null   float64
 2   volatile acidity      6489 non-null   float64
 3   citric acid           6494 non-null   float64
 4   residual sugar        6495 non-null   float64
 5   chlorides             6495 non-null   float64
 6   free sulfur dioxide   6497 non-null   float64
 7   total sulfur dioxide  6497 non-null   float64
 8   density               6497 non-null   float64
 9   pH                    6488 non-null   float64
 10  sulphates             6493 non-null   float64
 11  alcohol               6497 non-null   float64
 12  quality               6497 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


###결측치 날리기

In [42]:
df= df.dropna()
df.isna().sum(axis=0)


type                    0
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [43]:
df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


###2. 테스트 데이터 분리

In [89]:
x.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9


In [46]:
y.head()

0    6
1    6
2    6
3    6
4    6
Name: quality, dtype: int64

In [86]:
# 인덱스르 0부터로 땡기기
# 3-9 --> 0-6으로 바꾸기
df['quality']=df['quality']-3
df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,3
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,3
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,3
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,3
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,3


In [87]:
df['quality'].value_counts().sort_index()

0      30
1     214
2    2128
3    2820
4    1074
5     192
6       5
Name: quality, dtype: int64

In [ ]:
y= df['quality']
x= df.drop(['type','quality'],axis= 1)

In [91]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train,y_test = train_test_split(x,y,test_size=0.2, stratify= y, random_state= 32190403)
x_train.shape, x_test.shape, y_train.shape,y_test.shape

((5170, 11), (1293, 11), (5170,), (1293,))

In [92]:
y_train.value_counts().sort_index()

# 마지막 인덱스의 수가 너무 적어서 오버샘플링을 해서 개수맞추기 필요
# 무작정 복사, 값의 중앙값을 계속 뽑아내서 채워나가는 방법,

0      24
1     171
2    1702
3    2256
4     859
5     154
6       4
Name: quality, dtype: int64

In [64]:
# U는 업데이트
!pip install -U imbalanced-learn

In [94]:
# 오버샘플링하기
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
x_train, y_train = ros.fit_resample(x_train, y_train)

In [95]:
y_train.value_counts().sort_index()

0    2256
1    2256
2    2256
3    2256
4    2256
5    2256
6    2256
Name: quality, dtype: int64

In [96]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()


In [97]:
# standard scaler 표준화 작업

x_train_s = ss.fit_transform(x_train)
# 넘파이 형식으로 바뀜

x_train_s[20]

array([-1.16947894,  0.15111861,  0.66165901, -0.12088779, -0.1714778 ,
        0.41813874,  2.19386945, -0.11414462,  0.04748906,  0.34458566,
       -0.95607133])

In [98]:
x_test_s = ss.transform(x_test)

###학습
- 베이스라인

In [99]:
from sklearn.ensemble import RandomForestClassifier

rfc= RandomForestClassifier()
rfc.fit(x_train_s, y_train)

RandomForestClassifier()

In [103]:
from sklearn.metrics import accuracy_score
y_pred= rfc.predict(x_test_s)

print('정확도:', accuracy_score(y_test, y_pred))
#정확도


정확도: 0.6991492652745553


####튜닝

In [104]:
%%time
# grid search

from sklearn.model_selection import GridSearchCV
params= {
    'max_depth':[50,100,150,200]
}

rfc= RandomForestClassifier()

grid_cv=GridSearchCV(rfc, param_grid=params,cv=5, n_jobs=-1)
# 위 코드와 다른게 모지
grid_cv.fit(x_train_s, y_train)



CPU times: user 2.54 s, sys: 95.2 ms, total: 2.63 s
Wall time: 38.6 s


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [50, 100, 150, 200]})

In [106]:
grid_cv.best_params_
# best params 뽑아보기

{'max_depth': 200}

In [108]:
#정확도 추출
grid_cv.best_estimator_.score(x_test_s, y_test)


0.7006960556844548

모델 비교하기

In [110]:
#knn
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

knn= KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
# k개수 5, 2의 power를 가진 minkowski거리 사용
knn.fit(x_train, y_train)

y_pred= knn.predict
accuracy= (y_test == y_pred).mean()
print("정확도 = %.2f" % accuracy)

testAcc = []
trainAcc = []
for k in range(1, 50): # k가 1부터 49까지
    # KNN 으로 Train 데이터 세트를 학습한다.
    knn = KNeighborsClassifier(n_neighbors=k, p=2, metric='minkowski')
    knn.fit(x_train, y_train)

    # Test 세트의 Feature에 대한 정확도
    y_pred = knn.predict(x_test)
    testAcc.append((y_test == y_pred).sum() / len(y_pred))

    # Train 세트의 Feature에 대한 정확도
    y_pred = knn.predict(x_train)
    trainAcc.append((y_train == y_pred).sum() / len(y_pred))
​
plt.figure(figsize=(8, 5))
plt.plot(testAcc, label="Test Data")
plt.plot(trainAcc, label="Train Data")
plt.legend()
plt.xlabel("k")
plt.ylabel("Accuracy")
plt.show()

SyntaxError: ignored

In [112]:
from sklearn.neighbors import KNeighborsClassifier

clf_knn = KNeighborsClassifier()
clf_knn.fit(x_train_s, y_train)


KNeighborsClassifier()

In [113]:
clf_knn.score(x_test_s,y_test)

0.5212683681361175

In [115]:
from sklearn.svm import SVC

clf_svc = SVC(kernel = 'linear')
clf_svc.fit(x_train_s, y_train)


SVC(kernel='linear')

In [116]:
clf_svc.score(x_test_s, y_test)

0.33874709976798145

In [117]:
from sklearn.tree import DecisionTreeClassifier

clf_dt =DecisionTreeClassifier()
clf_dt.fit(x_train_s, y_train)

DecisionTreeClassifier()

In [118]:
clf_dt.score(x_test_s, y_test)

0.6102088167053364

In [ ]:
#svc
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, dataset

# confusion matrix를 통한 정확도 확인

from sklearn.metrics import confusion_matrix    # confusion_matrix라이브러리
y_pred = clf.predict(x)                         # 학습데이터 분류예측
confusion_matrix(y, y_pred)

# LinearSVM 활용
clf = svm.LinearSVC(C=C, max_iter = 10000)              # 학습 반복횟수 10000
clf.fit(x,y)
y_pred = clf.predict(x)
confusion_matrix(y, y_pred)




In [ ]:
#decision tree

# 3가지 유형으로 하는 방법 정리해두기


In [ ]:
# 결론적으로 randomforest가 성능이 제일 잘나온다.